In [1]:
%load_ext autoreload

In [2]:
import sys
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['OPENAI_API_KEY'] = 'zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

sys.path.append('/local/nlp/milad/code/explainability-of-art-images/')


In [3]:
%autoreload

import utils

In [4]:
from share4v.model.builder import load_pretrained_model
from share4v.mm_utils import get_model_name_from_path
from share4v.eval.run_share4v import eval_model

model_path = "Lin-Chen/ShareGPT4V-7B"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

[2025-06-11 09:22:18,043] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/local/nlp/milad/miniconda3/envs/share4v/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/local/nlp/milad/minico

[2025-06-11 09:22:22,114] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


/local/nlp/milad/miniconda3/envs/share4v/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load vision tower from Lin-Chen/ShareGPT4V-7B_Pretrained_vit-large336-l12


In [21]:
from PIL import Image
import numpy as np
import os
import torch

def get_sharegpt4v_image_representations(folder_path, processor, model, device='cuda:0'):
    """
    Loads images from a folder, processes them using the ShareGPT4V processor
    to obtain pixel values, and then obtains their latent representations
    using the vision encoder part of the ShareGPT4V model.

    Args:
        folder_path (str): Path to the folder containing images.
        processor: The loaded ShareGPT4V processor.
        model: The loaded ShareGPT4V model.

    Returns:
        dict: A dictionary where keys are image file paths and values are
              the corresponding latent representations (PyTorch tensors on CPU).
    """
    representations = {}
    
    if not os.path.isdir(folder_path):
        print(f"Error: Folder not found at {folder_path}")
        return representations

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image = Image.open(image_path).convert("RGB") # PIL Image
            
            # Process the image using the ShareGPT4V processor
            # This typically handles normalization and conversion to pixel_values
            # The processor might be a LlavaProcessor or similar
            image_tensor = processor(image, return_tensors='pt')['pixel_values']
            pixel_values = image_tensor.to(device)

            with torch.no_grad():
                # ShareGPT4V models (LLaVA-based) usually have an 'encode_images' method
                # or access vision features through a 'vision_tower' or 'get_vision_tower()'
                if hasattr(model, 'encode_images'):
                    # This is a common pattern in LLaVA-style models
                    image_features = model.encode_images(pixel_values)
                elif hasattr(model, 'vision_tower') and callable(model.vision_tower):
                    # If vision_tower is the encoder itself
                    # Output might need selection, e.g., .last_hidden_state or specific pooler output
                    vision_tower_output = model.vision_tower(pixel_values)
                    if hasattr(vision_tower_output, 'last_hidden_state'):
                        image_features = vision_tower_output.last_hidden_state
                    elif isinstance(vision_tower_output, torch.Tensor): # Direct tensor output
                        image_features = vision_tower_output
                    else:
                        print(f"Warning: 'model.vision_tower' output type not directly usable for {filename}. Output: {type(vision_tower_output)}")
                        continue
                elif hasattr(model, 'get_vision_tower') and callable(model.get_vision_tower):
                    vision_tower = model.get_vision_tower()
                    vision_tower_output = vision_tower(pixel_values)
                    if hasattr(vision_tower_output, 'last_hidden_state'):
                        image_features = vision_tower_output.last_hidden_state
                    elif isinstance(vision_tower_output, torch.Tensor):
                            image_features = vision_tower_output
                    else:
                        print(f"Warning: 'model.get_vision_tower()(...)' output type not directly usable for {filename}. Output: {type(vision_tower_output)}")
                        continue
                else:
                    print(f"Error: Could not find a suitable method (encode_images, vision_tower) to extract image features for {filename}")
                    continue
            
            # The output of encode_images or vision_tower might be [batch_size, num_tokens, hidden_size]
            # or [batch_size, hidden_size] if pooled.
            # For consistency, let's assume we want the features before any projection LLaVA might do.
            # If image_features has 3 dims (B, N, D) and you need a pooled version (B,D),
            # you might need to average or take a specific token (e.g., CLS token if applicable).
            # For now, we store what the vision encoder returns.
            representations[image_path] = image_features.cpu() # Store on CPU
    return representations


In [22]:
reps = get_sharegpt4v_image_representations('./data/sample_images', image_processor, model.model)

AttributeError: 'CLIPImageProcessor' object has no attribute 'image_processor'

In [24]:
dir(image_processor)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_class',
 '_create_repo',
 '_get_files_timestamps',
 '_processor_class',
 '_set_processor_class',
 '_upload_modified_files',
 'center_crop',
 'crop_size',
 'do_center_crop',
 'do_convert_rgb',
 'do_normalize',
 'do_rescale',
 'do_resize',
 'feature_extractor_type',
 'from_dict',
 'from_json_file',
 'from_pretrained',
 'get_image_processor_dict',
 'image_mean',
 'image_processor_type',
 'image_std',
 'model_input_names',
 'normalize',
 'preprocess',
 'push_to_hub',
 'register_for_auto_class',
 'resample',
 'rescale',
 'rescale_factor',
 'resize',
 'save_pretrained',
 'size',
 'to_dict',
 'to_json_file',
